In [ ]:
import cv2

def drawBoundingBox(img, bbox):
    # Extract bounding box coordinates
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    # Draw a rectangle around the object
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Add "Tracking" label near the bounding box
    cv2.putText(img, "Tracking", (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

cap = cv2.VideoCapture(0)

# tracker = cv2.TrackerKCF_create()    # loses tracking when object is out of frame and when the object is moving fast
# tracker = cv2.legacy.TrackerBoosting_create() # slower than mosse but more accurate
# tracker = cv2.legacy.TrackerMOSSE_create() # supposed to be faster than CSRT but less accurate
tracker = cv2.TrackerCSRT_create()    # Better fps when tracking moving objects than boosting
# tracker = cv2.legacy.TrackerMedianFlow_create()  # the bounding box will be larger when the object is moving fast
# tracker = cv2.legacy.TrackerTLD_create() # not accurate at all

success, img = cap.read()

bbox = cv2.selectROI("Select Region of Interest", img, False)
tracker.init(img, bbox)

while True:
    timer = cv2.getTickCount()  
    success, img = cap.read()  
    success, bbox = tracker.update(img)  


    if success:
        drawBoundingBox(img, bbox)
    else:
        
        cv2.putText(img, "Lost", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)

    # Calculate and display FPS
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(img, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    
    cv2.imshow("Track using different trackers", img)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
